# SteamSpy Data Cleaning

![steam_logo](https://buy.thewitcher.com/img/shops/steam.svg)



Welcome to the final part of the data cleaning process. Once we're finished here we'll be ready to move on to exploring and analysing the data.

As a quick re-cap, so far we have downloaded information on games from the Steam Store via the Steam API and SteamSpy API. We have cleaned and processed the data from the Steam API, and in this section we'll walkthrough cleaning data downloaded from the SteamSpy API. The overall goal of this project is to collect, clean and analyse data from the Steam Store with the idea of advising a fictional game developer or company.



## Import Libraries and Inspect Data

Let's begin by importing the necessary libraries and inspecting the raw downloaded data.

In [11]:
# standard library imports
from ast import literal_eval
import itertools
# import time
# import re

# third-party imports
import numpy as np
import pandas as pd

# customisations
pd.set_option("max_columns", 100)

In [12]:
raw_steamspy_data = pd.read_csv('./data/download/steamspy_data.csv')
raw_steamspy_data.head()

,appid,name,developer,publisher,score_rank,positive,negative,userscore,owners,average_forever,average_2weeks,median_forever,median_2weeks,price,initialprice,discount,languages,genre,ccu,tags
0,10,Counter-Strike,Valve,Valve,NaN,144330,3754,0,"20,000,000 .. 50,000,000",12988,287,166,36,999.0,999.0,0.0,"English, French, German, Italian, Spanish - Sp...",Action,18041,"{'Action': 5284, 'FPS': 4685, 'Multiplayer': 3..."
1,20,Team Fortress Classic,Valve,Valve,NaN,3804,711,0,"2,000,000 .. 5,000,000",129,143,36,143,499.0,499.0,0.0,"English, French, German, Italian, Spanish - Sp...",Action,95,"{'Action': 722, 'FPS': 290, 'Multiplayer': 243..."
2,30,Day of Defeat,Valve,Valve,NaN,3909,468,0,"5,000,000 .. 10,000,000",184,143,28,143,499.0,499.0,0.0,"English, French, German, Italian, Spanish - Spain",Action,196,"{'FPS': 774, 'World War II': 241, 'Multiplayer..."
3,40,Deathmatch Classic,Valve,Valve,NaN,1469,317,0,"5,000,000 .. 10,000,000",36,143,15,143,499.0,499.0,0.0,"English, French, German, Italian, Spanish - Sp...",Action,9,"{'Action': 622, 'FPS': 132, 'Classic': 101, 'M..."
4,50,Half-Life: Opposing Force,Gearbox Software,Valve,NaN,6529,359,0,"5,000,000 .. 10,000,000",197,143,121,143,499.0,499.0,0.0,"English, French, German, Korean",Action,202,"{'FPS': 860, 'Action': 280, 'Classic': 229, 'S..."


There are a lot of columns here that will overlap with columns we already dealt with in the Steam data. This is fine because it means we don't have to clean them here, and can simply use the columns from the other dataset.

A few columns in this data that could be valuable are `positive`, `negative`, `userscore`, `owners` and `average_forever`. We'll begin by looking at the null counts for each column.

In [13]:
raw_steamspy_data.isnull().sum()

appid                  0
name                   7
developer            177
publisher            168
score_rank         33052
positive               0
negative               0
userscore              0
owners                 0
average_forever        0
average_2weeks         0
median_forever         0
median_2weeks          0
price                 29
initialprice          22
discount              22
languages             94
genre                210
ccu                    0
tags                   0
dtype: int64

Most of the columns look fine, with only a few tens or hundreds of missing values, or none at all. The only column we'll remove for certain is the `score_rank` column, as it has far too many null values to be useful.

## Handling missing values

There are some columns we'll be removing, as they're present in the other dataset. Before we do we can investigate the missing values, to decide whether we should remove these rows first. We don't need to go through every column, but just look at a few individual ones.

In the `name` column, we have five rows with missing data, and four more rows where the name is 'none'. We'll certainly remove all 9 of these rows.

In [14]:
raw_steamspy_data[raw_steamspy_data['name'].isnull()]

,appid,name,developer,publisher,score_rank,positive,negative,userscore,owners,average_forever,average_2weeks,median_forever,median_2weeks,price,initialprice,discount,languages,genre,ccu,tags
479,17760,NaN,NaN,NaN,NaN,0,0,0,"0 .. 20,000",0,0,0,0,0.0,0.0,0.0,NaN,NaN,0,[]
7234,396420,NaN,NaN,Aterdux Entertainment,NaN,22,10,0,"20,000 .. 50,000",1,0,1,0,0.0,0.0,0.0,"English, Not supported",NaN,0,"{'Free to Play': 24, 'Action': 21, 'Indie': 21..."
9519,460250,NaN,Jeroen Wimmers,Jeroen Wimmers,NaN,48,4,0,"0 .. 20,000",0,0,0,0,899.0,899.0,0.0,"English, French, Italian, German, Spanish - Sp...","Casual, Indie",0,"{'Indie': 35, 'Casual': 34, 'Puzzle': 16, 'Min..."
13700,576960,NaN,Nuclear Tales,NaN,NaN,66,13,0,"0 .. 20,000",0,0,0,0,1499.0,1499.0,0.0,"English, Spanish - Spain, French, German","Adventure, Casual, Indie, RPG, Simulation, Str...",3,"{'Adventure': 35, 'Simulation': 33, 'Strategy'..."
21965,806160,NaN,Paleno Games,Paleno Games,NaN,4,19,0,"0 .. 20,000",0,0,0,0,99.0,99.0,0.0,"English, French, Italian, German, Spanish - Sp...","Action, Adventure, Casual, Indie",0,"{'Action': 22, 'Indie': 22, 'Casual': 21, 'Adv..."
27227,965340,NaN,2nd Studio,2nd Studio,NaN,44,2,0,"0 .. 20,000",0,0,0,0,199.0,199.0,0.0,"English, German, Danish, Japanese, Russian, Si...","Action, Indie, Simulation",1,"{'Indie': 33, 'Sexual Content': 32, 'Action': ..."
30315,1071920,NaN,Ultimo Games,Ultimo Games,NaN,14,0,0,"0 .. 20,000",0,0,0,0,399.0,399.0,0.0,English,"Action, Indie",0,"{'Action': 32, 'Indie': 32, 'Retro': 14, ""Shoo..."


In [15]:
raw_steamspy_data[raw_steamspy_data['name'] == 'none']

,appid,name,developer,publisher,score_rank,positive,negative,userscore,owners,average_forever,average_2weeks,median_forever,median_2weeks,price,initialprice,discount,languages,genre,ccu,tags
4921,339860,none,NaN,Green Sauce Games,NaN,84,33,0,"50,000 .. 100,000",356,0,389,0,0.0,0.0,0.0,English,NaN,0,"{'Indie': 53, 'RPG': 33, 'Adventure': 32, 'Mas..."
6784,385020,none,none,Valsar,NaN,13,27,0,"20,000 .. 50,000",272,0,276,0,0.0,0.0,0.0,"English, French, Italian, German, Spanish - Sp...","Casual, Indie, Massively Multiplayer, Strategy",0,"{'Indie': 47, 'Strategy': 31, 'Casual': 21, 'M..."
7348,398970,none,none,none,NaN,10,21,0,"50,000 .. 100,000",270,0,273,0,0.0,0.0,0.0,English,"Adventure, Indie, Massively Multiplayer, RPG, ...",0,"{'Massively Multiplayer': 38, 'Strategy': 23, ..."


There are 177 rows with missing developer data, and 140 rows with missing publisher data. It's conceivable that a small studio may self-publish, but we'll say that a title must have a developer for it to be kept. We can remove the rows with missing developer data, but then we'll be removing both of these columns as this information is already provided in the Steam data.

In [16]:
raw_steamspy_data[raw_steamspy_data['developer'].isnull()].sample(5, random_state=0)

,appid,name,developer,publisher,score_rank,positive,negative,userscore,owners,average_forever,average_2weeks,median_forever,median_2weeks,price,initialprice,discount,languages,genre,ccu,tags
2058,240220,Dead Island Riptide JP,NaN,NaN,NaN,0,0,0,"0 .. 20,000",0,0,0,0,0.0,0.0,0.0,NaN,NaN,4,[]
19395,737230,Rising Storm 2: Vietnam CTB Beta,NaN,NaN,NaN,0,0,0,"0 .. 20,000",19,0,22,0,NaN,NaN,NaN,NaN,NaN,0,[]
2112,243060,Urban Chaos,NaN,"Square Enix, Eidos Interactive",NaN,101,46,0,"20,000 .. 50,000",0,0,0,0,0.0,0.0,0.0,English,NaN,1,"{'Action': 36, 'Classic': 14, 'Female Protagon..."
2065,240660,Rain Blood Chronicles: Mirage,NaN,Origo Games,NaN,105,46,0,"0 .. 20,000",0,0,0,0,499.0,499.0,0.0,English,NaN,0,"{'Action': 35, 'Indie': 31, 'Adventure': 22, '..."
11976,530940,BIOHAZARD 7 resident evil グロテスクVer.,NaN,NaN,NaN,0,0,0,"0 .. 20,000",0,0,0,0,NaN,NaN,NaN,NaN,NaN,0,[]


In [17]:
raw_steamspy_data[raw_steamspy_data['publisher'].isnull()].sample(5)

,appid,name,developer,publisher,score_rank,positive,negative,userscore,owners,average_forever,average_2weeks,median_forever,median_2weeks,price,initialprice,discount,languages,genre,ccu,tags
4869,338710,Urja,Breaking Fourth,NaN,NaN,87,82,0,"20,000 .. 50,000",367,0,476,0,999.0,999.0,0.0,English,"Casual, Indie, Strategy",0,"{'Strategy': 34, 'Casual': 28, 'Indie': 26, 'P..."
2051,239490,America's Army: Proving Grounds Beta (Closed),NaN,NaN,NaN,0,0,0,"0 .. 20,000",0,0,0,0,NaN,0.0,0.0,NaN,NaN,0,[]
7074,391920,Ether One Redux,NaN,NaN,NaN,0,0,0,"0 .. 20,000",0,0,0,0,NaN,0.0,0.0,NaN,NaN,0,[]
12162,536670,Fear Of Traffic,Flavio Rausa,NaN,NaN,2,1,0,"0 .. 20,000",0,0,0,0,699.0,699.0,0.0,English,"Action, Casual, Racing",0,"{'Action': 21, 'Casual': 21, 'Racing': 21}"
28332,1003690,CrossTrix,Jason Free,NaN,NaN,1,0,0,"0 .. 20,000",0,0,0,0,999.0,999.0,0.0,English,"Casual, Indie, Strategy",0,"{'Casual': 22, 'Indie': 20, 'Strategy': 19}"


There are a small number of rows with missing price data. This is another column we don't need, (as well as `initialprice` and `discount`), as we already have that data available in the other dataset. 

It looks like these rows have a lot of other missing information as well, so if they haven't already been removed when cleaning other columns we'll remove them here.

In [18]:
raw_steamspy_data[raw_steamspy_data['price'].isnull()].sample(5)

,appid,name,developer,publisher,score_rank,positive,negative,userscore,owners,average_forever,average_2weeks,median_forever,median_2weeks,price,initialprice,discount,languages,genre,ccu,tags
15978,648590,428 〜封鎖された渋谷で〜,NaN,NaN,NaN,0,0,0,"0 .. 20,000",0,0,0,0,NaN,NaN,NaN,NaN,NaN,0,[]
10719,499450,The Witcher 3: Wild Hunt Game of the Year Edition,NaN,NaN,NaN,0,0,0,"0 .. 20,000",0,0,0,0,NaN,NaN,NaN,NaN,NaN,0,[]
7074,391920,Ether One Redux,NaN,NaN,NaN,0,0,0,"0 .. 20,000",0,0,0,0,NaN,0.0,0.0,NaN,NaN,0,[]
10666,497960,Legends of Callasia Demo,NaN,NaN,NaN,0,0,0,"0 .. 20,000",0,0,0,0,NaN,NaN,NaN,NaN,NaN,0,[]
17997,698920,王国出击,NaN,NaN,NaN,0,0,0,"0 .. 20,000",0,0,0,0,NaN,NaN,NaN,NaN,NaN,0,[]


Again there are a small number of rows with missing language data. There are a couple of bigger titles here, but most are missing other data also. We'll go ahead and remove these rows.

In [19]:
raw_steamspy_data[raw_steamspy_data['languages'].isnull()].sample(5, random_state=0)

,appid,name,developer,publisher,score_rank,positive,negative,userscore,owners,average_forever,average_2weeks,median_forever,median_2weeks,price,initialprice,discount,languages,genre,ccu,tags
249,8350,Strong Bad's Cool Game for Attractive People: ...,Telltale Games,Telltale Games,NaN,18,6,0,"0 .. 20,000",1,0,1,0,0.0,0.0,0.0,NaN,Adventure,1,{'Adventure': 21}
1230,94500,Back to the Future: Ep 2 - Get Tannen!,Telltale Games,Telltale Games,NaN,119,16,0,"0 .. 20,000",127,0,168,0,0.0,0.0,0.0,NaN,NaN,1,"{'Point & Click': 9, 'Adventure': 6}"
2115,243180,The Harvest,NaN,NaN,NaN,0,0,0,"0 .. 20,000",0,0,0,0,0.0,0.0,0.0,NaN,NaN,0,[]
754,34390,Football Manager 2011 Demo,NaN,NaN,NaN,0,0,0,"0 .. 20,000",0,0,0,0,0.0,0.0,0.0,NaN,NaN,0,[]
566,23140,KrissX,NaN,NaN,NaN,0,0,0,"0 .. 20,000",0,0,0,0,0.0,0.0,0.0,NaN,NaN,1,[]


## Remove unwanted columns

As mentioned, there are a number of columns present in the Steam dataset, so we can remove any of those with crossover. It makes sense that these exist because SteamSpy's data is retrieved by looking at Steam's data. We'll remove the `score_rank` column as it contains too many missing values, and we'll remove the `userscore` column as most rows have a value of 0 here, so it won't be useful.

In [20]:
raw_steamspy_data['userscore'].value_counts().head()

0      33052
100        4
84         3
95         3
68         2
Name: userscore, dtype: int64

There are also three columns - `average_2weeks`, `median_2weeks`, (average and median playtime in the last 2 weeks) and `ccu` (Peak conccurrent users yesterday) - that provide information only relevant to the time the data was collected. We will thus remove those columns.

In [21]:
drop_cols = [
    'score_rank', # too many missing values
    'userscore', # too little variance (most have 0)
    'genre', 'developer', 'publisher', 'price', 'initialprice', 'discount', # provided by Steam data
    'average_2weeks', 'median_2weeks', 'ccu' # not interested in temporally specific columns
]

One more column we could remove is the `languages` column, as we already those processed in the Steam Data. Here in the SteamSpy data, languages is made up of a comma-separated list of all supported languages. This provides us with some more information so we can keep it for now.

In [22]:
raw_steamspy_data['languages'].head()

0    English, French, German, Italian, Spanish - Sp...
1    English, French, German, Italian, Spanish - Sp...
2    English, French, German, Italian, Spanish - Spain
3    English, French, German, Italian, Spanish - Sp...
4                      English, French, German, Korean
Name: languages, dtype: object

## Process & Export Tags Column

There are a couple of columns that require more specific handling, and the `tags` column is one of them. There will be a lot of overlap with the `genres` column, but the tags might provide just enough unique information to be worthwhile keeping.

Below we separate out the tags data for experimentation, then have a look a the first few rows. Each tag for a game has an associated number, corresponding to the amount of votes that game has for that tag. A game's tags with higher votes means that more people think it is a relevant tag for that title.

In [23]:
tags = raw_steamspy_data['tags']

print(tags[0])
tags.head()

{'Action': 5284, 'FPS': 4685, 'Multiplayer': 3260, 'Shooter': 3234, 'Classic': 2692, 'Team-Based': 1778, 'First-Person': 1633, 'Competitive': 1532, 'Tactical': 1282, "1990's": 1137, 'e-sports': 1129, 'PvP': 819, 'Old School': 706, 'Military': 597, 'Strategy': 591, 'Masterpiece': 560, 'Survival': 283, 'Score Attack': 268, '1980s': 239, 'Assassin': 215}


0    {'Action': 5284, 'FPS': 4685, 'Multiplayer': 3...
1    {'Action': 722, 'FPS': 290, 'Multiplayer': 243...
2    {'FPS': 774, 'World War II': 241, 'Multiplayer...
3    {'Action': 622, 'FPS': 132, 'Classic': 101, 'M...
4    {'FPS': 860, 'Action': 280, 'Classic': 229, 'S...
Name: tags, dtype: object

Just by looking at the first row you can see that there are probably going to be far too many tags to include without disrupting the dataset. We have a couple of options here. We could include just the names of the top 3 or 5 tags for a game - which should be easy enough as it looks like the tags are already in order from most votes to least - or we could create new columns for each tag, and the number of votes as the row values, like this:

appid | name | action | fps | multiplayer
--- | --- | --- | --- | ---
10 | Shoot 'em up | 2681 | 2048 | 0
20 | Big Brawl | 208  | 0 | 172


In a way, we'll do both.

Because including the full tag information in the dataset may make it bulky and unwieldy, we'll only include the top three tags in the core dataset. As we did when cleaning the Steam Data, we'll also export the full tag data as a separate file. This means that if we want to use it as part of our analysis we can import and merge the data quite easily.

Before we can do that, we have to figure out how to handle and process the data. As we've seen previously, the data in each row is stored as a string containing a dictionary, so we have to use `literal_eval` to get python to "recognise" the dictionary first.

In [24]:
eval_row = literal_eval(tags[0])

print(eval_row['Action'])
print(eval_row['FPS'])

5284
4685


We also have to figure out how to get a list of all the different unique tags from across the dataset. As each row only contains the tags that it has votes for, we don't know from the outset the tag names or even how many different tags there are.

We could loop through the data and build a list of all the unique tag names using a dictionary or something similar, but instead we'll achieve the same thing much more easily using the [chain](https://docs.python.org/3/library/itertools.html?highlight=itertools#itertools.chain) function from itertools. As seen below, it will take a variety of iterables and unpack them into a single sequence (we have to use list() here to force the generator to evaluate). We can then use the set function to obtain just the unique values from the returned sequence.

As you can see below, we can pass lists, tuples and dictionaries and they will all be handled. Also notice how only the key from the dictionary is returned (d) and not the value inside the key (e). This is really useful and means we can just pass the evaluated tags data as a series, and itertools will just extract the individual keys.

In [25]:
values = [
    ['a', 'b'], # list
    ('b', 'c'), # tuple
    {'d': 'e'}  # dictionary
]

list(itertools.chain(*values))

['a', 'b', 'b', 'c', 'd']

In [26]:
set(itertools.chain(*values))

{'a', 'b', 'c', 'd'}

If we tried to define a function now, we might run into a problem. It turns out not all of the rows have data stored as dictionaries. Seen below, there are a few hundred rows with an empty list.

In [27]:
tags[tags == '[]'].shape[0]

480

This doesn't cause an issue when we generate our set of unique tag names, as any empty lists are ignored.

In [28]:
parsed_tags = tags.apply(lambda x: literal_eval(x))

cols = set(itertools.chain(*parsed_tags))

print('Number of unique tags:', len(cols))
print('\nFirst few tags:', sorted(list(cols))[:5])

Number of unique tags: 409

First few tags: ['1980s', "1990's", '2.5D', '2D', '2D Fighter']


However to create the new dataframe from the tags and votes, we need to look inside the dictionaries in each row. We'll expand the logic to parse the tags using the `isintance` function to check each row for its type. If it is a dictionary, we return the dictionary. If it is a list, we'll return an empty dictionary `{}`. We'll also include an extra check in case there is anything else in the rows we haven't found yet. 

Next, we loop through the tags stored in `cols`, standardise the format of the new column names, then extract the vote count of each tag by checking if the tag name is in each row's dictionary of tags.

In [29]:
def parse_tags(x):
    x = literal_eval(x)
    
    if isinstance(x, dict):
        return x
    elif isinstance(x, list):
        return {}
    else:
        raise TypeError('Something other than dict or list found')

parsed_tags = tags.apply(parse_tags)
        
tag_data = pd.DataFrame()

for col in sorted(cols):
    # standardise column names
    col_name = col.lower().replace(' ', '_').replace('-', '_').replace("'", "")

    # check if column in row's dictionary of tags and return that value if it is, or 0 if it isn't
    tag_data[col_name] = parsed_tags.apply(lambda x: x[col] if col in x.keys() else 0)

tag_data.head()

,1980s,1990s,2.5d,2d,2d_fighter,2d_platformer,360_video,3d,3d_fighter,3d_platformer,3d_vision,4_player_local,4x,6dof,atv,abstract,action,action_rpg,action_roguelike,action_adventure,addictive,adventure,agriculture,aliens,alternate_history,america,animation_&_modeling,anime,arcade,archery,arena_shooter,artificial_intelligence,assassin,asymmetric_vr,asynchronous_multiplayer,atmospheric,audio_production,auto_battler,automation,automobile_sim,bmx,base_building,baseball,based_on_a_novel,basketball,batman,battle_royale,beat_em_up,beautiful,benchmark,...,time_attack,time_management,time_manipulation,time_travel,top_down,top_down_shooter,touch_friendly,tower_defense,trackir,trading,trading_card_game,traditional_roguelike,trains,transhumanism,transportation,trivia,turn_based,turn_based_combat,turn_based_strategy,turn_based_tactics,tutorial,twin_stick_shooter,typing,underground,underwater,unforgiving,utilities,vr,vr_only,vampire,vehicular_combat,video_production,villain_protagonist,violent,visual_novel,voice_control,voxel,walking_simulator,war,wargame,warhammer_40k,web_publishing,werewolves,western,word_game,world_war_i,world_war_ii,wrestling,zombies,e_sports
0,239,1137,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5284,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,215,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1129
1,0,110,0,0,0,0,0,0,0,0,0,0,0,0,0,0,722,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,34,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,154,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,143,0,0,0,0,0,0,14,241,0,0,0
3,0,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,622,0,0,0,0,0,0,0,0,0,0,0,0,0,40,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,122,0,0,0,0,0,0,0,0,0,0,0,0,0,0,280,0,0,0,0,107,0,157,0,0,0,0,0,0,0,0,0,0,0,96,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


We'll have to add in the AppIDs, but apart from that this makes up the tag data that we'll export. Next we have to figure out how to extract just the top three tags for use in the core dataset.

It turns out this is a relatively simple process. As the keys are in order of highest votes to least, we can just extract the first 3 keys for each row. This may not be the safest approach, as dictionaries don't necessarily preserve key order, but it's fine for our purposes. We can then join these keys on a semicolon to create a list of the tags.

In [30]:
def parse_tags(x):
    x = literal_eval(x)

    if isinstance(x, dict):
        return ';'.join(list(x.keys())[:3])
    else:
        return np.nan
    
tags.apply(parse_tags).head()

0          Action;FPS;Multiplayer
1          Action;FPS;Multiplayer
2    FPS;World War II;Multiplayer
3              Action;FPS;Classic
4              FPS;Action;Classic
Name: tags, dtype: object

## Handle Owners Column

The final column we'll look at before defining a function to perform the cleaning is the `owners` column. This column is made up of the lower and upper bound of an estimation for the amount of owners for each title. For privacy reasons, SteamSpy can't get exact figures from Steam (though it [used to](https://www.polygon.com/2018/4/12/17229752/steam-spy-charts-new-privacy-rules-valve)), so we'll have to make do with the figures we have here.

In [31]:
owners = raw_steamspy_data['owners']
owners.head()

0    20,000,000 .. 50,000,000
1      2,000,000 .. 5,000,000
2     5,000,000 .. 10,000,000
3     5,000,000 .. 10,000,000
4     5,000,000 .. 10,000,000
Name: owners, dtype: object

We will reformat the data slightly. This way, we will be able to choose in our exploration and exploitation part how to process those values.

In [32]:
owners.str.replace(',', '').str.replace(' .. ', '-').head()

0    20000000-50000000
1      2000000-5000000
2     5000000-10000000
3     5000000-10000000
4     5000000-10000000
Name: owners, dtype: object

## Define Function

We're now ready to define the functions to clean the data. We'll wrap all the logic inside a `process` function, and for most of the columns there isn't much code to write so we can confine it here. The only logic we'll separate out is for processing the tags column, as it is a more complex endeavour.

In the next code cell we clean the data, export the full tags data to file, and inspect the clean dataframe. After that we inspect the exported data, verifying everything went as planned.

In [33]:
def process_tags(df, export=False):
    if export: 
        
        tag_data = df[['appid', 'tags']].copy()
        
        def parse_export_tags(x):
            x = literal_eval(x)

            if isinstance(x, dict):
                return x
            elif isinstance(x, list):
                return {}
            else:
                raise TypeError('Something other than dict or list found')

        tag_data['tags'] = tag_data['tags'].apply(parse_export_tags)

        cols = set(itertools.chain(*tag_data['tags']))

        for col in sorted(cols):
            col_name = col.lower().replace(' ', '_').replace('-', '_').replace("'", "")

            tag_data[col_name] = tag_data['tags'].apply(lambda x: x[col] if col in x.keys() else 0)

        tag_data = tag_data.drop('tags', axis=1)

        tag_data.to_csv('./data/exports/steamspy_tag_data.csv', index=False)
        print("Exported tag data to './data/exports/steamspy_tag_data.csv'")
        
        
    def parse_tags(x):
        x = literal_eval(x)
        
        if isinstance(x, dict):
            return ';'.join(list(x.keys())[:3])
        else:
            return np.nan
    
    df['tags'] = df['tags'].apply(parse_tags)
    
    # rows with null tags seem to be superseded by newer release, so remove (e.g. dead island)
    df = df[df['tags'].notnull()]
    
    return df


def process(df):
    df = df.copy()
    
    # handle missing values
    df = df[(df['name'].notnull()) & (df['name'] != 'none')]
    df = df[df['developer'].notnull()]
    df = df[df['languages'].notnull()]
    df = df[df['price'].notnull()]
    
    # remove unwanted columns
    df = df.drop([
        'genre', 'developer', 'publisher', 'score_rank', 'userscore', 'average_2weeks',
        'median_2weeks', 'price', 'initialprice', 'discount', 'ccu'
    ], axis=1)
    
    # keep top tags, exporting full tag data to file
    df = process_tags(df, export=True)
    
    # reformat owners column
    df['owners'] = df['owners'].str.replace(',', '').str.replace(' .. ', '-')
    
    return df


steamspy_data = process(raw_steamspy_data)
steamspy_data.head()

Exported tag data to './data/exports/steamspy_tag_data.csv'


,appid,name,positive,negative,owners,average_forever,median_forever,languages,tags
0,10,Counter-Strike,144330,3754,20000000-50000000,12988,166,"English, French, German, Italian, Spanish - Sp...",Action;FPS;Multiplayer
1,20,Team Fortress Classic,3804,711,2000000-5000000,129,36,"English, French, German, Italian, Spanish - Sp...",Action;FPS;Multiplayer
2,30,Day of Defeat,3909,468,5000000-10000000,184,28,"English, French, German, Italian, Spanish - Spain",FPS;World War II;Multiplayer
3,40,Deathmatch Classic,1469,317,5000000-10000000,36,15,"English, French, German, Italian, Spanish - Sp...",Action;FPS;Classic
4,50,Half-Life: Opposing Force,6529,359,5000000-10000000,197,121,"English, French, German, Korean",FPS;Action;Classic


In [34]:
# inspect tag data
pd.read_csv('./data/exports/steamspy_tag_data.csv').head()

,appid,1980s,1990s,2.5d,2d,2d_fighter,2d_platformer,360_video,3d,3d_fighter,3d_platformer,3d_vision,4_player_local,4x,6dof,atv,abstract,action,action_rpg,action_roguelike,action_adventure,addictive,adventure,agriculture,aliens,alternate_history,america,animation_&_modeling,anime,arcade,archery,arena_shooter,artificial_intelligence,assassin,asymmetric_vr,asynchronous_multiplayer,atmospheric,audio_production,auto_battler,automation,automobile_sim,bmx,base_building,baseball,based_on_a_novel,basketball,batman,battle_royale,beat_em_up,beautiful,...,time_attack,time_management,time_manipulation,time_travel,top_down,top_down_shooter,touch_friendly,tower_defense,trackir,trading,trading_card_game,traditional_roguelike,trains,transhumanism,transportation,trivia,turn_based,turn_based_combat,turn_based_strategy,turn_based_tactics,tutorial,twin_stick_shooter,typing,underground,underwater,unforgiving,utilities,vr,vr_only,vampire,vehicular_combat,video_production,villain_protagonist,violent,visual_novel,voice_control,voxel,walking_simulator,war,wargame,warhammer_40k,web_publishing,werewolves,western,word_game,world_war_i,world_war_ii,wrestling,zombies,e_sports
0,10,239,1137,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5284,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,215,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1129
1,20,0,110,0,0,0,0,0,0,0,0,0,0,0,0,0,0,722,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,34,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,30,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,154,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,143,0,0,0,0,0,0,14,241,0,0,0
3,40,0,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,622,0,0,0,0,0,0,0,0,0,0,0,0,0,40,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,50,0,122,0,0,0,0,0,0,0,0,0,0,0,0,0,0,280,0,0,0,0,107,0,157,0,0,0,0,0,0,0,0,0,0,0,96,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


## Merge and Export Clean Data

Before we wrap up, we'll check for any missing values and export the SteamSpy data to its own csv file. Then we'll look at combining the previously cleaned Steam data and the newly cleaned SteamSpy data into one cohesive dataset. This will give us options and flexibility for analysis in the future, and means we will have one csv file to begin looking at.

In [35]:
steamspy_data.isnull().sum()

appid              0
name               0
positive           0
negative           0
owners             0
average_forever    0
median_forever     0
languages          0
tags               0
dtype: int64

In [36]:
steamspy_data.to_csv('./data/exports/steamspy_clean.csv', index=False)

To combine the dataframes we can use the pandas [merge](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.merge.html) function, specifying which columns to merge upon.

In [37]:
steam_data = pd.read_csv('./data/exports/steam_data_clean.csv')
    
merged = steam_data.merge(steamspy_data, left_on='steam_appid', right_on='appid', suffixes=('', '_steamspy'))
merged.head()

,name,steam_appid,required_age,platforms,categories,genres,achievements,release_date,price,english,french,german,developer,publisher,appid,name_steamspy,positive,negative,owners,average_forever,median_forever,languages,tags
0,Half-Life 2: Lost Coast,340,0,windows;mac;linux,Single-player;Commentary available;Remote Play...,Action,0,2005-10-27,0.0,1,1,1,Valve,Valve,340,Half-Life 2: Lost Coast,6412,1072,2000000-5000000,123,23,"English, French, German, Italian, Spanish - Sp...",FPS;Action;Singleplayer
1,Team Fortress 2,440,0,windows;mac;linux,Multi-player;Cross-Platform Multiplayer;Steam ...,Action;Free to Play,520,2007-10-10,0.0,1,1,1,Valve,Valve,440,Team Fortress 2,588020,37107,50000000-100000000,8895,452,"English, Danish, Dutch, Finnish, French, Germa...",Hero Shooter;Free to Play;Multiplayer
2,Dota 2,570,0,windows;mac;linux,Multi-player;Co-op;Steam Trading Cards;Steam W...,Action;Free to Play;Strategy,0,2013-07-09,0.0,1,1,1,Valve,Valve,570,Dota 2,1018172,174020,100000000-200000000,29989,1284,"Bulgarian, Czech, Danish, Dutch, English, Finn...",Free to Play;MOBA;Multiplayer
3,Alien Swarm,630,0,windows,Single-player;Multi-player;Co-op;Steam Achieve...,Action,66,2010-07-19,0.0,1,0,0,Valve,Valve,630,Alien Swarm,17946,974,1000000-2000000,319,96,English,Free to Play;Co-op;Action
4,Counter-Strike: Global Offensive,730,0,windows;mac;linux,Multi-player;Steam Achievements;Full controlle...,Action;Free to Play,167,2012-08-21,0.0,1,1,1,Valve;Hidden Path Entertainment,Valve,730,Counter-Strike: Global Offensive,3295593,470639,20000000-50000000,25886,7129,"Czech, Danish, Dutch, English, Finnish, French...",FPS;Shooter;Multiplayer


Finally, we can remove, reorder and rename some columns before exporting the complete, clean dataset.

We'll remove the SteamSpy rows with overlap - name, languages and AppID. Next we reindex the dataframe, reordering the columns so it makes a little more logical sense to look at, and then we rename some of the columns to make it a bit clearer what they describe. 

In [38]:
# remove overlapping columns
steam_clean = merged.drop(['name_steamspy', 'languages', 'steam_appid'], axis=1)

# reindex to reorder columns
steam_clean = steam_clean[[
    'appid',
    'name',
    'release_date',
    'english',
    'french',
    'german',
    'developer',
    'publisher',
    'platforms',
    'required_age',
    'categories',
    'genres',
    'tags',
    'achievements',
    'positive',
    'negative',
    'average_forever',
    'median_forever',
    'owners',
    'price'
]]

steam_clean = steam_clean.rename({
    'tags': 'steamspy_tags',
    'positive': 'positive_ratings',
    'negative': 'negative_ratings',
    'average_forever': 'average_playtime',
    'median_forever': 'median_playtime'
}, axis=1)

steam_clean.head()

,appid,name,release_date,english,french,german,developer,publisher,platforms,required_age,categories,genres,steamspy_tags,achievements,positive_ratings,negative_ratings,average_playtime,median_playtime,owners,price
0,340,Half-Life 2: Lost Coast,2005-10-27,1,1,1,Valve,Valve,windows;mac;linux,0,Single-player;Commentary available;Remote Play...,Action,FPS;Action;Singleplayer,0,6412,1072,123,23,2000000-5000000,0.0
1,440,Team Fortress 2,2007-10-10,1,1,1,Valve,Valve,windows;mac;linux,0,Multi-player;Cross-Platform Multiplayer;Steam ...,Action;Free to Play,Hero Shooter;Free to Play;Multiplayer,520,588020,37107,8895,452,50000000-100000000,0.0
2,570,Dota 2,2013-07-09,1,1,1,Valve,Valve,windows;mac;linux,0,Multi-player;Co-op;Steam Trading Cards;Steam W...,Action;Free to Play;Strategy,Free to Play;MOBA;Multiplayer,0,1018172,174020,29989,1284,100000000-200000000,0.0
3,630,Alien Swarm,2010-07-19,1,0,0,Valve,Valve,windows,0,Single-player;Multi-player;Co-op;Steam Achieve...,Action,Free to Play;Co-op;Action,66,17946,974,319,96,1000000-2000000,0.0
4,730,Counter-Strike: Global Offensive,2012-08-21,1,1,1,Valve;Hidden Path Entertainment,Valve,windows;mac;linux,0,Multi-player;Steam Achievements;Full controlle...,Action;Free to Play,FPS;Shooter;Multiplayer,167,3295593,470639,25886,7129,20000000-50000000,0.0


# Transfer it to the exploration part AFTER !!! 

# Economic Succes

Before going further and finalizing our data processing, we will add an other metric to define the economic success of our game. This metric will only have a meaning on paid game in this dataset. Indeed, we do not have any information if the so-called free to play game have or not adds or in game items that you need to actually buy with money. 


This metric is only relevant for the games that are bought from the steam library. 

In [39]:
steam_clean['economic_success'] = [price*np.array(list(map(int, owners.split('-')))).mean() for price, owners   in zip(steam_clean['price'], steam_clean['owners'])]

In [40]:
# export clean dataset
steam_clean.to_csv('./data/steam_clean.csv', index=False)
len(steam_clean)

29380

And with that, we have come to the end of the cleaning process. It's been quite a long journey and now we can look ahead to analysing the data, creating some visualisations, and gaining some insights into the multi-billion dollar industry that is video games.